In [80]:
import os
import math
from collections import Counter, defaultdict, OrderedDict
from itertools import chain
from functools import reduce
import re
import random
import numpy as np
import scipy

import matplotlib.pyplot as plt
%matplotlib inline

## DATASET PATH

In [17]:
train_dir = 'datas/train/'
test_dir = 'datas/test/'
val_dir = 'datas/valid/'

def geo_mean(values):
    return math.exp(1.0 / len(values) * sum([math.log(p) for p in values])) 

## Probability Construction

### $\{pi_i\}$ initial state probability of state $i$
### $A=\{a_{ij}\}$. $a_{ij}$ state transaction probability from $i$ to $j$
### $B=\{b_{ik}\}$. $b_{ik}$ symbol emission probability from $i$ to $k$

In [4]:
bos = '<BOS>/bos'
eos  ='<EOS>/eos'

def word_split(line):
    if '/l/%' in line:  # A strange annotation bug
        line = line.replace('/l/%', '/l')
    words = [bos] + line.split() + [eos]  # append bos, eos
    states = [word.split('/')[1] for word in words]
    states_pair = list(zip(states[:-1], states[1:]))  # (state i, state j)
    states_head = states[:-1]
    state_word_pair = [tuple(word.split('/')[::-1]) for word in words]  # (state i, word j)
    return states_pair, state_word_pair, states_head, states

# word_split('年根儿/n  了/y  ，/w  一张张/m  贺卡/n  ，/w  飞越/v  关山/n 。/m ')

In [5]:
def hmm_model(dataset_path):
    """
    return A, B (\pi is included in A)
    """
    states_pairs = []
    state_word_pairs = []
    state_heads = []
    states = []
    for dir_name in os.listdir(dataset_path):
        with open(dataset_path + dir_name, 'r') as file:
            for line in file.readlines():
                states_pair, state_word_pair, state_head, state = word_split(line)
                states_pairs.append(states_pair)
                state_word_pairs.append(state_word_pair)
                state_heads.append(state_head)
                states.append(state)

    states_pair_counter = Counter(chain(*states_pairs))
    state_word_pair_counter = Counter(chain(*state_word_pairs))
    state_head_counter = Counter(chain(*state_heads))
    state_counter = Counter(chain(*states))
    sum_state_head = sum(state_head_counter.values())
    sum_counter = sum(state_counter.values())

    states_pair_prob = defaultdict(lambda: 1.0 / sum_state_head, {states_pair: counter / state_head_counter[states_pair[0]]
                        for states_pair, counter in states_pair_counter.items()})
    state_word_pair_prob = defaultdict(lambda: 1.0 / sum_counter, {state_word_pair: counter / state_counter[state_word_pair[0]]
                            for state_word_pair, counter in state_word_pair_counter.items()})

    return states_pair_prob, state_word_pair_prob, state_counter


In [7]:
states_pair_prob, state_word_pair_prob, state_counter = hmm_model(train_dir)

## HMM
$$
p(\mathbf{W}|\mu) = \sum_{\mathbf{C}\in \mathcal{P}^{\mathcal{C}}} p(\mathbf{C}|\mu)p(\mathbf{W}|\mathbf{C}) = \sum_{\mathbf{C}\in \mathcal{P}^{\mathcal{C}}} p(c_1|\mu)\prod_{i=1}^{N} p(c_{i+1}|c_i)p(w_i|c_i)
$$

### forward hmm
$$
\alpha(i) = \pi{i}\\
a_i(t+1) = \sum_{j=1}^{N_c}a_j(t)a_{ij}b_{j,w_{t}}\\
p(\mathbf{W}|\mu) = \sum_{j=1}^{N_c} a_j(T+1)
$$

In [92]:
def forward_hmm_prob(states_pair_prob, state_word_pair_prob, states, sentence_line):
    _, state_word_pair, _, _ = word_split(sentence_line)
    alpha = {state: 0 for state in states}
    alpha['bos'] = 1.0
    for t in range(1, len(state_word_pair)):
        _, word = state_word_pair[t]
        alpha_new = dict()
        for state in states:            
            a  = [alpha[old_state] * states_pair_prob[(old_state, state)]
                                    for old_state in alpha]
            alpha_new[state] = sum(a) *  state_word_pair_prob[(state, word)]
            
        alpha = alpha_new
    return sum(alpha.values())


def forward_hmm_prob_logsumexp(states_pair_prob, state_word_pair_prob, states, sentence_line):
    _, state_word_pair, _, _ = word_split(sentence_line)
    states = list(states)
    alpha = np.array([states_pair_prob[('bos', state)] * state_word_pair_prob[(state_word_pair[1])] for state in states], dtype=np.double)
    
    for t in range(2, len(state_word_pair)):
        _, word = state_word_pair[t]
        alpha_new = np.zeros_like(alpha)
        for idx, state in enumerate(states):
            prob = np.array([states_pair_prob[(old_state, state)] for old_state in states], dtype=np.double)
            alpha_new[idx] = np.exp(scipy.misc.logsumexp(np.log(alpha) + np.log(prob))) * state_word_pair_prob[state_word_pair[t]]
        alpha = alpha_new
    return np.sum(alpha)

def hmm_test(dataset_path, hmm_prob_fn):
    perplexity_dict = dict()
    def perplexity(line):
        prob = hmm_prob_fn(line)
        _, words, _, _ = word_split(line)
        if prob == 0:
            print('!!!!!!!')
            return 0
        return np.exp(-1.0/len(words) * math.log(prob))
    for dir_name in os.listdir(dataset_path):
        print(dir_name)
        with open(dataset_path + dir_name, 'r') as file:
            for line in file.readlines():
                perplexity_dict[line] = perplexity(line)
    return perplexity_dict

In [93]:
sent = '谢谢/v  ！/w  （/w  新华社/nt  北京/ns  １２月/t  ３１日/t  电/n  ）/w  '
sent1 = '该/r  商场/n  有/v  １０７/m  部/q  电梯/n  ，/w  其中/r  有/v  自动/b  扶梯/n  、/w  观光/vn  梯/Ng  、/w  货梯/n  、/w  直达/vn  或/c  跨/v  层/n  梯/Ng  等等/u  。/w  值得一提/l  的/u  是/v  ，/w  “/w  东安/nz  ”/w  地下/s  停车场/n  拥有/v  ５００/m  个/q  车位/n  。/w  这/r  在/p  “/w  寸土寸金/l  ”/w  的/u  王府井/ns  十分/m  难得/a  又/d  十分/m  必要/a  。/w  '
print(forward_hmm_prob_logsumexp(states_pair_prob, state_word_pair_prob, list(state_counter.keys()), sent))
print(forward_hmm_prob(states_pair_prob, state_word_pair_prob, list(state_counter.keys()), sent))


4.39693935494e-19
3.3273675675762847e-29


In [97]:
val_perplexity_dict = hmm_test(val_dir, lambda line: forward_hmm_prob(states_pair_prob, state_word_pair_prob, list(state_counter.keys()), line))
geo_mean([ x for x in val_perplexity_dict.values() if x != 0])

01240910.txt
01241001.txt
01241002.txt
!!!!!!!
!!!!!!!
01241003.txt
!!!!!!!
!!!!!!!
!!!!!!!
01241004.txt
!!!!!!!
!!!!!!!
01241005.txt
!!!!!!!
!!!!!!!
01241006.txt
!!!!!!!
!!!!!!!
01241007.txt
!!!!!!!
!!!!!!!
01241008.txt
!!!!!!!
!!!!!!!
01241009.txt
!!!!!!!
!!!!!!!
01241010.txt
!!!!!!!
!!!!!!!
01241011.txt
!!!!!!!
!!!!!!!
!!!!!!!
01241012.txt
!!!!!!!
01241101.txt
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
01241102.txt
!!!!!!!
!!!!!!!
!!!!!!!
01241103.txt
!!!!!!!
!!!!!!!
01241104.txt
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
01241105.txt
!!!!!!!
01241106.txt
!!!!!!!
!!!!!!!
!!!!!!!
01241107.txt
!!!!!!!
01241108.txt
01241201.txt
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
01241202.txt
01241203.txt
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
01241204.txt
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
01241205.txt
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
01241206.txt
01241207.txt
01241208.txt
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
01241209.txt
01250101.txt
01250102.txt
!!!!!!!
!!!!!!!


1315.1410753117414